In [ ]:
!pip install pickle5
!pip install keras-tuner

     |████████████████████████████████| 133kB 16.5MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219244 sha256=7f9885c44e88a57b6613a2c41c5c63bf8339bab8a18d5a0a122f8b2e07431246
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
     |████████████████████████████████| 71kB 5.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=b9845f78cf85a9c457ddfa6f3f68abbbfe6f0ebfd3b46b5602df45a389949676
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=e0491b14fd6386ab935c1c5bf3a38a10b3a1d4bd07e5da0a043bf760dab716b0
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import pickle5 as pickle

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
X_train = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SCAM/X_train_fSC.pkl')
y_train = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SCAM/y_train_fSC.pkl')
X_test = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SCAM/X_test_fSC.pkl')
y_test = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SCAM/y_test_fSC.pkl')

fpath1 = '/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Glove SCAM/X_train_glSC.pkl'
fpath2 = '/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Glove SCAM/X_test_glSC.pkl'

with open(fpath1, "rb") as fh:
  X_train_glove = pickle.load(fh)
with open(fpath2, "rb") as fh:
  X_test_glove = pickle.load(fh)

glove_word_vector_matrix = np.load('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Glove SCAM/glove_matrixSC.npy')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout, SimpleRNN, LSTM, Bidirectional,Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, SpatialDropout1D

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator # Generates batches for sequence data

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l1_l2, l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch, Hyperband
from sklearn.metrics import confusion_matrix

In [ ]:
max_length = X_train_glove.shape[1]
vocab_size = glove_word_vector_matrix.shape[0]

def configure_matrix(choice):
    if choice == 'glove':
        embedding = glove_word_vector_matrix
    elif choice == 'cc':
        embedding = cc_word_vector_matrix
    else:
        print ('Select a valid embedding.')
    return embedding

embedding = configure_matrix('glove')

def build_model(hp):
    model = Sequential()
    model.add(Embedding(vocab_size, 300, input_length = max_length, weights= [embedding], trainable=False))
    
    model.add(SpatialDropout1D(hp.Float('spatial_dropout', min_value = 0.1, max_value = 0.5, step = 0.1)))
    
    for i in range(hp.Int('num_bdlstm_layers', 2, 3, default = 2)):
        model.add(Bidirectional
                  (LSTM
                   (units = hp.Int('bdlstm1_units_' + str(i), min_value = 32, max_value = 128, step = 16),
                    return_sequences = True,
                    activation = hp.Choice('activation1_' + str(i), values = ['relu', 'tanh'], default = 'tanh'),
                    recurrent_dropout = hp.Float('recurrent_dropout1_' + str(i), min_value = 0, max_value = 0.5, step = 0.1))
                    )
                   )
        
        model.add(Dropout(
            rate = hp.Float('dropout_rate_' + str(i), min_value = 0, max_value = 0.6, step = 0.2))
            )
        
    model.add(Bidirectional
                (LSTM
                 (units = hp.Int('bdlstm2_units', min_value = 32, max_value = 128, step = 16),
                  return_sequences = False,
                  activation = hp.Choice('activation2_', values = ['relu', 'tanh'], default = 'tanh'),
                  recurrent_dropout = hp.Float('recurrent_dropout2_' + str(i), min_value = 0, max_value = 0.5, step = 0.1)))
                  )
                 
                
    for j in range(hp.Int('num_dense_layers', 1, 2, default = 1)):
        model.add(Dense
                  (units=hp.Int('dense_units_' + str(j), min_value = 12, max_value = 24, step=12),
                   activation='relu')
                  )
        
    model.add(Dense(1, activation='sigmoid'))
    optimizer =  hp.Choice('optimizer', values = ['adam', 'rmsprop'])
    if optimizer == 'adam':      
        model.compile(
            optimizer = Adam(
                learning_rate = ExponentialDecay(
                    initial_learning_rate = hp.Choice('learning_rate', values =[1e-2, 1e-3, 1e-4]),
                    decay_steps = hp.Choice('decay_steps', values = [50000, 100000, 200000, 500000]),
                    decay_rate = hp.Choice('decay_rate', values = [1e-7, 1e-6, 1e-5]),
                    staircase = True)
                ),
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])
    elif optimizer == 'rmsprop':
        model.compile(
            optimizer = RMSprop(
                learning_rate = ExponentialDecay(
                    initial_learning_rate = hp.Choice('learning_rate', values =[1e-2, 1e-3, 1e-4]),
                    decay_steps = hp.Choice('decay_steps', values = [50000, 100000, 200000, 500000]),
                    decay_rate = hp.Choice('decay_rate', values = [1e-7, 1e-6, 1e-5]),
                    staircase = True)
                ),
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])
    
    return model

embeddings = {'Glove' : [],
              'CC' : []}

def tune(X_train, y_train, name):

    tuner = Hyperband(build_model,
                      objective='val_accuracy',
                      max_epochs = 200,
                        factor=3,
                        directory= 'BDLSTM_Spam',
                        project_name = name,
                        seed = 1)

    tuner.search_space_summary()
    
    tuner.search(X_train, y_train,
                 callbacks = [EarlyStopping(monitor='val_loss', patience=5)],
                 validation_split= 0.25)
    
    best_model = tuner.get_best_models(num_models=1)[0]
    
    return best_model


In [ ]:
embeddings = {'Glove' : [],
              'CC' : []}
              
embeddings['Glove'] = tune(X_train_glove, y_train, 'Glove')

Trial 47 Complete [00h 03m 53s]
val_accuracy: 0.7605262994766235

Best val_accuracy So Far: 0.9789473414421082
Total elapsed time: 03h 04m 22s

Search: Running Trial #48

Hyperparameter    |Value             |Best Value So Far 
spatial_dropout   |0.4               |0.3               
num_bdlstm_layers |2                 |2                 
bdlstm1_units_0   |96                |64                
activation1_0     |tanh              |tanh              
recurrent_dropo...|0.2               |0                 
dropout_rate_0    |0.2               |0.2               
bdlstm1_units_1   |80                |128               
activation1_1     |relu              |tanh              
recurrent_dropo...|0.2               |0.5               
dropout_rate_1    |0.4               |0.6               
bdlstm2_units     |80                |32                
activation2_      |tanh              |tanh              
recurrent_dropo...|0.2               |0.4               
num_dense_layers  |2           

KeyboardInterrupt: ignored